In [4]:
import torch
import torchvision
import torch.optim as optim
from data.data import get_train_test_loader
from model.network import FastUpdateNet, FastUpdateNetLargeEMNIST, TeacherNetLargeEMNIST
import torch.nn.functional as F

import pickle
import time 

In [5]:
n_epochs = 16
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
# torch.manual_seed(random_seed)

In [6]:
train_loader, test_loader = get_train_test_loader('emnist')

In [7]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [8]:
from threading import Thread

def compute_back_M1(network):
  network.mNet1.backwardHidden()

def compute_back_M2(network):
  network.mNet2.backwardHidden()

def train(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    # p1 = Thread(target=compute_back_M1, args=[network]) # start two independent threads
    # p2 = Thread(target=compute_back_M2, args=[network])
    
    # p1.start()
    # p2.start()
    network.mNet1.backwardHidden()
    network.mNet2.backwardHidden()
    # p1.join() # wait for the two threads to finish
    # return 
    # p2.join()

    correctness = (target == torch.argmax(output))
    optimizer.step()
    # network.mNet.weightUpdate(correctness, lr = learning_rate)
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))


In [9]:
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracies.append(100. * correct / len(test_loader.dataset))
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [10]:
def train_teacherNet(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    correctness = (target == torch.argmax(output))
    optimizer.step()
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracies.append(100. * correct / len(test_loader.dataset))
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

## Test using 2 Ms (computing grad in main thread and separate thread)

In [11]:
net = None
optimizer = None
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []
net_types = [ 'M']
results = {}
trials = 5
for nt in net_types:
  results[nt] = []
  for i in range(trials):
    print('model type', nt, 'trail', i, 'starts')
    train_losses = []
    train_counter = []
    test_losses = []
    test_accuracies = []
    if nt == 'M':
      net = FastUpdateNetLargeEMNIST()
      optimizer = optim.SGD(net.get_parameters(), lr=learning_rate, momentum=momentum)
    else:
      net = TeacherNetLargeEMNIST()
      optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
    for epoch in range(1, n_epochs + 1):
      if nt == 'M':
        train(epoch, net)
      else:
        train_teacherNet(epoch, net)
      test(net)
    results[nt].append({'train_losses':train_losses, 'train_counter':train_counter, 'test_losses':test_losses, 'test_accuracies':test_accuracies})

torch.save(results, './EMNIST-result.pt')

model type M trail 0 starts


/home/hao/Documents/magic-m/model/network.py:253: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(o_5)
/home/hao/miniconda/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 3.8508, Accuracy: 400/18800 (2%)


Test set: Avg. loss: 3.8502, Accuracy: 400/18800 (2%)


Test set: Avg. loss: 3.8501, Accuracy: 399/18800 (2%)


Test set: Avg. loss: 3.8500, Accuracy: 493/18800 (3%)


Test set: Avg. loss: 3.8499, Accuracy: 457/18800 (2%)


Test set: Avg. loss: 3.8497, Accuracy: 400/18800 (2%)


Test set: Avg. loss: 3.8490, Accuracy: 460/18800 (2%)


Test set: Avg. loss: 3.8184, Accuracy: 695/18800 (4%)


Test set: Avg. loss: 3.2182, Accuracy: 1639/18800 (9%)


Test set: Avg. loss: 2.7653, Accuracy: 3620/18800 (19%)


Test set: Avg. loss: 2.4635, Accuracy: 4327/18800 (23%)


Test set: Avg. loss: 2.3604, Accuracy: 4671/18800 (25%)


Test set: Avg. loss: 2.2773, Accuracy: 5179/18800 (28%)


Test set: Avg. loss: 2.3232, Accuracy: 5111/18800 (27%)


Test set: Avg. loss: 2.2799, Accuracy: 5467/18800 (29%)


Test set: Avg. loss: 2.2496, Accuracy: 5720/18800 (30%)

model type M trail 1 starts

Test set: Avg. loss: 3.8508, Accuracy: 400/18800 (2%)


Test

In [12]:
import torch

net_types = ['base', 'M']
results = torch.load('./EMNIST-result.pt')
for nt in net_types:
    total = 0
    all_r = []
    for r in results[nt]:
        total = total + r['test_accuracies'][-1]
        all_r.append(r['test_accuracies'][-1])
    print(nt, total / len(results[nt]), torch.std(torch.tensor(all_r), dim = 0))

KeyError: 'base'

## Test using no Ms, and only on main thread

In [ ]:
def train_no_M(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    # p1 = Thread(target=compute_back_M1, args=[network]) # start two independent threads
    # # p2 = Thread(target=compute_back_M2, args=[network])
    
    # p1.start()
    # # p2.start()
    # network.mNet2.backwardHidden()
    # p1.join() # wait for the two threads to finish
    # # return 
    # # p2.join()

    correctness = (target == torch.argmax(output))
    optimizer.step()
    # network.mNet.weightUpdate(correctness, lr = learning_rate)
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [ ]:

import torch.nn as nn
fNet = FastUpdateNetLarge()
fNet.mNet1 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
fNet.mNet2 = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
optimizer = optim.SGD(fNet.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

s = time.perf_counter()

test(fNet)
for epoch in range(1, n_epochs + 1):
  train_no_M(epoch, fNet)
  test(fNet)
  # if (epoch + 1) % 10 == 0:
  #   torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
  #   with open('fNet-stdp-3000-epoch-loss-accuracy.pkl', 'wb') as f:
  #     pickle.dump({'train_losses':train_losses, 'test_losses': test_losses, 'test_accuracies':test_accuracies}, f, protocol=pickle.HIGHEST_PROTOCOL)

# torch.save(fNet, 'fNet-stdp-30000-epoch.pt')
e = time.perf_counter()
print(f"Training time for {epoch} epochs: {e-s} seconds")


Test set: Avg. loss: 2.3051, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 2.3021, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 2.2995, Accuracy: 1000/10000 (10%)


Test set: Avg. loss: 1.6677, Accuracy: 2402/10000 (24%)


Test set: Avg. loss: 0.8659, Accuracy: 6525/10000 (65%)


Test set: Avg. loss: 1.0468, Accuracy: 6144/10000 (61%)

Training time for 5 epochs: 49.55497955996543 seconds
